<a href="https://colab.research.google.com/github/ErycaFMS/DS_Projeto/blob/main/Projeto_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Definição do Problema

O dataset utilizado neste projeto é proveniente do sistema SCADA de uma turbina eólica localizada na Turquia. O objetivo é analisar a performance da turbina, monitorando e avaliando a produção de energia e as condições ambientais. Este dataset foi coletado em intervalos de 10 minutos e contém informações cruciais para o entendimento da eficiência da turbina e do impacto das condições do vento sobre a geração de energia. Para mais detalhes sobre este dataset, consulte: https://www.kaggle.com/datasets/berkerisen/wind-turbine-scada-dataset?resource=download&select=T1.csv

**Informações sobre os atributos:**

1. **Date/Time**: Data e hora em que a medição foi realizada (intervalos de 10 minutos).
2. **ActivePower** (kW): Potência ativa gerada pela turbina no momento da medição, em quilowatts (kW).
3. **Wind Speed** (m/s): Velocidade do vento na altura do eixo da turbina, medida em metros por segundo (m/s). Esta é a velocidade do vento utilizada para a geração de eletricidade.
4. **Theoretical_Power_Curve** (KWh): Valores teóricos de potência que a turbina pode gerar com a velocidade do vento fornecida, conforme especificado pelo fabricante da turbina, medidos em quilowatt-hora (KWh).
5. **Wind Direction** (°): Direção do vento na altura do eixo da turbina, medida em graus (°). As turbinas eólicas ajustam-se automaticamente para se alinhar com esta direção.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Carga de Dados

Iremos usar o pacote Pandas (Python Data Analysis Library) para carregar de um arquivo .csv sem cabeçalho disponível online.

Com o dataset carregado, iremos explorá-lo um pouco.

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/ErycaFMS/DS_Projeto/main/T1.csv"

# Informa o cabeçalho das colunas
colunas = ['Data/Hora', 'Potência Ativa [kW]', 'Velocidade do Vento [m/s]', 'Potência Teórica [kW]', 'Direção do Vento [°]']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')

# Converte a coluna 'Data/Hora' de string para datetime
dataset['Data/Hora'] = pd.to_datetime(dataset['Data/Hora'], format='%d %m %Y %H:%M')

In [ ]:
dataset.head()

## 3. Análise de Dados

### 3.1. Estatísticas Descritivas

Vamos iniciar examinando as dimensões do dataset, suas informações e alguns exemplos de linhas.

In [ ]:
# Mostra as dimensões do dataset
print(dataset.shape)

In [ ]:
# Mostra as informações do dataset
print(dataset.info())

In [ ]:
# Mostra as 10 primeiras linhas do dataset
dataset.head(10)

In [ ]:
# Mostra as 10 últimas linhas do dataset
dataset.tail(10)

É sempre importante verificar o tipo do atributos do dataset, pois pode ser necessário realizar conversões. Já fizemos anteriormente com o comando info, mas vamos ver uma outra forma de verificar a natureza de cada atributo e então exibir um resumo estatístico do dataset.

In [ ]:
# Verifica o tipo de dataset de cada atributo
dataset.dtypes

In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
dataset.describe()

Notamos que o valor mínimo de Potência Ativa é negativo, evidenciando futura necessidade de tratamento de dados.

### 3.2. Visualizações Unimodais

Vamos criar agora um histograma para cada atributo do dataset. Veremos que o atributo VelVento segue uma distribuição aproximadamente normal.

In [ ]:
# Histograma
dataset.hist(figsize = (15,10))
plt.show()

O Gráfico de Densidade, ou Density Plot, é bem parecido com o histograma, mas com uma visualização um pouco diferente. Com ele, pode ser mais fácil identificar a distribuição do atributos do dataset. Assim como fizemos com o histograma, vamos criar um density plot para cada atributo do dataset.

Veremos que muitos dos atributos têm uma distribuição distorcida. Uma transformação como a Box-Cox, que pode aproximar a distribuição de uma Normal, pode ser útil neste caso.

In [ ]:
# Remove a coluna 'Data/Hora' para o plot de densidade
dataset_numerico = dataset.drop(columns=['Data/Hora'])

# Density Plot
dataset_numerico.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

Vamos agora trabalhar com boxplots. No **boxblot**, a linha no centro (vermelha) representa o valor da mediana (segundo quartil ou p50). A linha abaixo é o 1o quartil (p25) e a linha acima o terceiro quartil (p75). O boxplot ajuda a ter uma ideia da dispersão dos dataset e os possíveis outliers.

*OBS: Se um ponto do dataset é muito distante da média (acima de 3 desvios padrão da média), pode ser considerado outlier.*

Nos gráficos bloxplot, veremos que a dispersão dos atributos do dataset é bem diferente.

In [ ]:
# Boxplot
dataset.plot(kind = 'box', subplots = True, layout = (3,3), sharex = False, sharey = False, figsize = (15,10))
plt.show()

### 3.3. Visualizações Multimodais

Ao visualizar as correlações entre os atributos através da matriz de correlação, perceberemos que parece haver alguma estrutura na ordem dos atributos. O azul ao redor da diagonal sugere que os atributos que estão próximos um do outro são geralmente mais correlacionados entre si. Os vermelhos também sugerem alguma correlação negativa moderada, a medida que os atributos

Vamos agora verificar a covariância entre as variáveis numéricas do dataset. A **covariância** representa como duas variáveis numéricas estão relacionadas. Existem várias formas de calcular a correlação entre duas variáveis, como por exemplo, o coeficiente de correlação de Pearson, que pode ser:
* Próximo de -1 : há uma correlação negativa entre as variáveis,
* Próximo de +1: há uma correlação positiva entre as variáveis.
* 0: não há correlação entre as variáveis.

<i>OBS: Esta informação é relevante porque alguns algoritmos como regressão linear e regressão logística podem apresentar problemas de performance se houver atributos altamente correlacionados. Vale a pena consultar a documentação do algoritmo para verificar se algum tipo de tratamento de dataset é necessário.</i>

Falamos anteriormente da importância da correlação entre os atributos, e agora iremos visualizar esta informação em formato gráfico. A **matriz de correlação** exibe graficamente a correlação entre os atributos numéricos do dataset.estão mais distantes um do outro na ordenação.

O código a seguir exibe a matriz de correlação.

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
sns.heatmap(dataset.corr(), annot=True, cmap='RdBu');

Por sua vez, o gráfico de dispersão (**scatter plot**) mostra o relacionamento entre duas variáveis. Vamos exibir um para cada par de atributos dos dataset, usando o Seaborn.

In [ ]:
# Scatter Plot com Seaborn - Variação 1

sns.pairplot(dataset)

## 4. Preparação dos dados

Nesta etapa, poderíamos realizar diversas operações de preparação de dados, como por exemplo, tratamento de valores missings (faltantes), limpeza de dados, transformações como one-hot-encoding, seleção de características (feature selection), entre outras não mostradas neste notebook. Lembre-se de não criar uma versão padronizada/normalizada dos dados neste momento (apesar de serem operações de pré-processamento) para evitar o Data Leakage.

### 4.1. Tratamento de Missings e Limpeza

Devemos realizar uma tratamento de dados para os valores de Potência que aparecem negativos, além dos missings. Vamos então fazer este tratamento e criar uma nova visão do nosso dataset.

In [ ]:
# verificando nulls no dataset
dataset.isnull().sum()

In [ ]:
#Criando cópia do Dataset
DatasetAntes = dataset.copy()

In [ ]:
# salvando um NOVO dataset para tratamento de missings (para não sobrescrever o dataset original foi criado DatasetAntes)

# recuperando os nomes das colunas
col = list(DatasetAntes.columns)

# substituindo os zeros por NaN
DatasetAntes.replace(0, np.nan, inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(DatasetAntes)

print(DatasetAntes.info())

In [ ]:
# removendo a coluna Data/Hora
DatasetAntes.drop(['Data/Hora'], axis=1, inplace= True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(DatasetAntes)



Gerando o gráfico de Potência Ativa e Velocidade do vento, para fins comparativos.

In [ ]:
#Plotando o gráfico potência ativa x velocidade do vento
plt.figure(figsize=(15, 10))  # Define o tamanho da figura
sns.pairplot(DatasetAntes, x_vars='Velocidade do Vento [m/s]', y_vars='Potência Ativa [kW]', height=6)  # Ajusta o tamanho dos gráficos
plt.show()

In [ ]:
# Criando Pipeline de Limpeza de dados:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer

# Função para substituir valores negativos por zero
def correct_negative_values(X):
    X['Potência Ativa [kW]'] = X['Potência Ativa [kW]'].apply(lambda x: max(x, 0))
    return X


In [ ]:
# Função para interpolar valores faltantes e suavizar outliers
def interpolate_and_smooth(X):
    X['Potência Ativa [kW]'] = X['Potência Ativa [kW]'].interpolate(method='polynomial', order=2)
    X['Velocidade do Vento [m/s]'] = X['Velocidade do Vento [m/s]'].interpolate(method='polynomial', order=2)
    return X

    # Definindo as colunas numéricas e categóricas
numeric_features = ['Potência Ativa [kW]', 'Velocidade do Vento [m/s]', 'Potência Teórica [kW]']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar valores faltantes com a média
    ('scaler', StandardScaler())  # Escalar os dados numéricos
])

categorical_features = ['Direção do Vento [°]']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar valores categóricos com o mais frequente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificar as categorias
])

# Combinando as transformações com as funções personalizadas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Definindo a pipeline completa
pipeline = Pipeline(steps=[
    ('negative_correction', FunctionTransformer(correct_negative_values)),
    ('interpolation', FunctionTransformer(interpolate_and_smooth)),
    ('preprocessor', preprocessor)
])

# Aplicação da pipeline no DataFrame
DatasetAntes = pd.DataFrame(DatasetAntes)
pipeline.fit_transform(dataset)

Verificando o gráfico antes x depois

In [ ]:
# Aplicar a pipeline ao dataset
dataset_limpo = pipeline.fit_transform(DatasetAntes)

# Como o resultado da pipeline é um array numpy, precisamos recriar um DataFrame
# Para isso, mantemos as mesmas colunas que foram transformadas
dataset_limpo = pd.DataFrame(dataset_limpo, columns=numeric_features + list(pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features)))

# Plotando o gráfico potência ativa x velocidade do vento após limpeza
plt.figure(figsize=(15, 10))  # Define o tamanho da figura
sns.pairplot(dataset_limpo, x_vars='Velocidade do Vento [m/s]', y_vars='Potência Ativa [kW]', height=6)  # Ajusta o tamanho dos gráficos
plt.show()

### 4.2. Separação em conjunto de treino e conjunto de teste

É uma boa prática usar um conjunto de teste (na literatura também chamado de conjunto de validação), uma amostra dos dados que não será usada para a construção do modelo, mas somente no fim do projeto para confirmar a precisão do modelo final. É um teste que podemos usar para verificar o quão boa foi a construção do modelo, e para nos dar uma ideia de como o modelo irá performar nas estimativas em dados não vistos. Usaremos 80% do conjunto de dados para modelagem e guardaremos 20% para teste, usando a estratégia train-test-split, já explicada anteriormente. Primeiramente, iremos sinalizar quais são as colunas de atributos (X - 0 a 7) e qual é a coluna das classes (Y - 8). Em seguida, especificaremos o tamanho do conjunto de teste desejado e uma semente (para garantir a reprodutibilidade dos resultados). Finalmente, faremos a separação dos conjuntos de treino e teste através do comando train_test_split, que retornará 4 estruturas de dados: os atributos e classes para o conjunto de teste e os atributos e classes para o conjunto de treino.


In [ ]:
# Selecionando as variáveis
X = dataset[['Velocidade do Vento [m/s]', 'Potência Teórica [kW]', 'Direção do Vento [°]']]
y = dataset['Potência Ativa [kW]']

# Separar os dados em treino e teste (80% para treino, 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4.3. Definindo os Modelos e Parâmetros

In [ ]:
# Definindo os modelos e seus hiperparâmetros
modelos_params = {
    'Regressão Linear': {
        'modelo': LinearRegression(),
        'params': {}  # Regressão Linear não tem hiperparâmetros para ajuste básico
    },
    'Árvore de Regressão': {
        'modelo': DecisionTreeRegressor(),
        'params': {
            'classifier__max_depth': [None, 5, 10],
            'classifier__min_samples_split': [2, 5, 10]
        }
    },
    'SVR': {
        'modelo': SVR(),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['linear', 'rbf'],
            'classifier__epsilon': [0.1, 0.2, 0.5]
        }
    },
    'KNN Regressor': {
        'modelo': KNeighborsRegressor(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'Gradient Boosting Regressor': {
        'modelo': GradientBoostingRegressor(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 0.2],
            'classifier__max_depth': [3, 5, 7]
        }
    }
}

### 4.4. Executando o Grid Search com Validação Cruzada

In [ ]:
# Definir o pré-processador (padronização)
preprocessor = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Lista para armazenar resultados
resultados = []

# Executando o Grid Search com validação cruzada para cada modelo
for nome, mp in modelos_params.items():
    modelo = mp['modelo']
    params = mp['params']

    # Criando o pipeline com o modelo atual
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])

    # Grid Search com validação cruzada
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    # Treinando o modelo
    grid_search.fit(X_train, y_train)

    # Melhor modelo encontrado
    best_model = grid_search.best_estimator_

    # Fazendo previsões no conjunto de teste
    y_pred = best_model.predict(X_test)

    # Calculando o erro quadrático médio (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Armazenando os resultados
    resultados.append({
        'Modelo': nome,
        'Melhores Hiperparâmetros': grid_search.best_params_,
        'Erro Quadrático Médio (MSE)': mse,
        'Pipeline': best_model  # Armazenando o melhor modelo
    })

# Exibindo os resultados
for resultado in resultados:
    print(f"Modelo: {resultado['Modelo']}")
    print(f"Melhores Hiperparâmetros: {resultado['Melhores Hiperparâmetros']}")
    print(f"Erro Quadrático Médio (MSE): {resultado['Erro Quadrático Médio (MSE)']}\n")

### 4.5. Comparando os Modelos Otimizados

In [ ]:
# Comparando os modelos otimizados
df_resultados = pd.DataFrame(resultados)

# Ordenando os resultados pelo menor Erro Quadrático Médio (MSE)
df_resultados = df_resultados.sort_values(by='Erro Quadrático Médio (MSE)', ascending=True)

# Resetando o índice
df_resultados.reset_index(drop=True, inplace=True)

# Exibindo as colunas desejadas
df_resultados[['Modelo', 'Erro Quadrático Médio (MSE)', 'Melhores Hiperparâmetros']]

### 4.6. Selecionando o melhor modelo

### 4.7. Salvando o modelo

### 5. Carregando o modelo e fazendo previsões

## Conclusão

Escreva aqui os seu principais achados, pontos de atenção e conclusões desse projeto.

## 10. Referências

- [Documentação do Scikit-Learn sobre Pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline)
- [Documentação do GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Salvando Modelos com Joblib](https://scikit-learn.org/stable/modules/model_persistence.html)
- [Dataset Titanic no Kaggle](https://www.kaggle.com/c/titanic/data)
- [Validação Cruzada no Scikit-Learn](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Métricas de Avaliação de Classificação](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)